In [ ]:
!pip install rohub

In [ ]:

import rohub
import sys
import os
import pandas as pd

module_path = os.path.abspath(os.path.join('..', 'linear-elastic-plate-with-hole'))

if module_path not in sys.path:
    sys.path.append(module_path)
import importlib
import provenance
from generate_config import workflow_config

importlib.reload(provenance)

In [3]:
USE_DEVELOPMENT_VERSION = False

In [4]:
if USE_DEVELOPMENT_VERSION:
    rohub.settings.API_URL = "https://rohub2020-rohub.apps.paas-dev.psnc.pl/api/"
    rohub.settings.KEYCLOAK_CLIENT_ID = "rohub2020-cli"
    rohub.settings.KEYCLOAK_CLIENT_SECRET = "714617a7-87bc-4a88-8682-5f9c2f60337d"
    rohub.settings.KEYCLOAK_URL = "https://keycloak-dev.apps.paas-dev.psnc.pl/auth/realms/rohub/protocol/openid-connect/token"
    rohub.settings.SPARQL_ENDPOINT = "https://rohub2020-api-virtuoso-route-rohub.apps.paas-dev.psnc.pl/sparql/"

In [ ]:
username = ""
password = ""

rohub.login(username=username, password=password)

In [21]:
ANNOTATION_PREDICATE = "http://w3id.org/nfdi4ing/metadata4ing#investigates"
ANNOTATION_OBJECT = "https://github.com/BAMresearch/NFDI4IngModelValidationPlatform/tree/main/benchmarks/linear-elastic-plate-with-hole"

In [ ]:
UUID_QUERY = f"""
SELECT ?subject 
WHERE {{
  ?subject <{ANNOTATION_PREDICATE}> <{ANNOTATION_OBJECT}> .
}}
"""

uuid_result = rohub.query_sparql_endpoint(UUID_QUERY)
uuids = []

if not uuid_result.empty:
    uuids = [iri.split('/')[-1] for iri in uuid_result["subject"]]
    print("UUIDs:", uuids)
else:
    uuids = []
    print("No results found")


In [ ]:
named_graphs = {}

for UUID in uuids:
    NAMED_GRAPH_QUERY = f"""
    PREFIX schema: <http://schema.org/>
    SELECT ?graph WHERE {{
        GRAPH ?graph {{ <https://w3id.org/ro-id/{UUID}> a schema:Dataset . }}
    }}
    """

    named_graph_result = rohub.query_sparql_endpoint(NAMED_GRAPH_QUERY)

    if not named_graph_result.empty:
        graph_iri = named_graph_result.iloc[0]["graph"]
        named_graphs[UUID] = graph_iri
        print(f"[{UUID}] Found Named Graph: {graph_iri}")
    else:
        named_graphs[UUID] = None
        print(f"[{UUID}] No named graph found")

In [53]:
analyzer = provenance.ProvenanceAnalyzer()

In [ ]:
parameters = ["element_size", "element_order", "element_degree"]
metrics = ["max_von_mises_stress_nodes"]
tools = workflow_config["tools"]

data = []

for uuid, named_graph in named_graphs.items():
    query_string = analyzer.build_dynamic_query(parameters, metrics, tools, named_graph)
    result = rohub.query_sparql_endpoint(query_string)
    if not result.empty:
        result["element_order"] = pd.to_numeric(
            result["element_order"], errors="coerce"
        )
        result["element_degree"] = pd.to_numeric(
            result["element_degree"], errors="coerce"
        )
        filtered_result = result[
            (result["element_order"] == 1) & (result["element_degree"] == 1)
        ]
        rows = [
            [
                float(row["element_size"]),
                float(row["max_von_mises_stress_nodes"]),
                row["tool_name"],
            ]
            for _, row in filtered_result.iterrows()
        ]
        data.extend(rows)

data.sort(key=lambda row: row[0], reverse=False)

analyzer.plot_provenance_graph(
    data=data,
    x_axis_label="Element Size",
    y_axis_label="Max Von Mises Stress Nodes",
    x_axis_index=0,
    y_axis_index=1,
    group_by_index=2,
    title="Element Size vs Max Von Mises Stress Nodes  \n element-order = 1 , element-degree = 1 ",
)

In [ ]:
parameters = ["element_size", "element_order", "element_degree"]
metrics = ["max_von_mises_stress_gauss_points"]
tools = workflow_config["tools"]

data = []

for uuid, named_graph in named_graphs.items():
    query_string = analyzer.build_dynamic_query(parameters, metrics, tools, named_graph)
    result = rohub.query_sparql_endpoint(query_string)
    if not result.empty:
        result["element_order"] = pd.to_numeric(
            result["element_order"], errors="coerce"
        )
        result["element_degree"] = pd.to_numeric(
            result["element_degree"], errors="coerce"
        )
        filtered_result = result[
            (result["element_order"] == 1) & (result["element_degree"] == 1)
        ]
        rows = [
            [
                float(row["element_size"]),
                float(row["max_von_mises_stress_gauss_points"]),
                row["tool_name"],
            ]
            for _, row in filtered_result.iterrows()
        ]
        data.extend(rows)

data.sort(key=lambda row: row[0], reverse=False)

analyzer.plot_provenance_graph(
    data=data,
    x_axis_label="Element Size",
    y_axis_label="Max Von Mises Stress Gauss Points",
    x_axis_index=0,
    y_axis_index=1,
    group_by_index=2,
    title="Element Size vs Max Von Mises Stress Gauss Points  \n element-order = 1 , element-degree = 1 ",
)

In [ ]:
parameters = ["element_size", "element_order", "element_degree"]
metrics = ["max_von_mises_stress_nodes"]
tools = workflow_config["tools"]

data = []

for uuid, named_graph in named_graphs.items():
    query_string = analyzer.build_dynamic_query(parameters, metrics, tools, named_graph)
    result = rohub.query_sparql_endpoint(query_string)
    if not result.empty:
        result["element_order"] = pd.to_numeric(
            result["element_order"], errors="coerce"
        )
        result["element_degree"] = pd.to_numeric(
            result["element_degree"], errors="coerce"
        )
        filtered_result = result[
            (result["element_order"] == 2) & (result["element_degree"] == 2)
        ]
        rows = [
            [
                float(row["element_size"]),
                float(row["max_von_mises_stress_nodes"]),
                row["tool_name"],
            ]
            for _, row in filtered_result.iterrows()
        ]
        data.extend(rows)

data.sort(key=lambda row: row[0], reverse=False)

analyzer.plot_provenance_graph(
    data=data,
    x_axis_label="Element Size",
    y_axis_label="Max Von Mises Stress Nodes",
    x_axis_index=0,
    y_axis_index=1,
    group_by_index=2,
    title="Element Size vs Max Von Mises Stress Nodes  \n element-order = 2 , element-degree = 2 ",
)

In [ ]:
parameters = ["element_size", "element_order", "element_degree"]
metrics = ["max_von_mises_stress_gauss_points"]
tools = workflow_config["tools"]

data = []

for uuid, named_graph in named_graphs.items():
    query_string = analyzer.build_dynamic_query(parameters, metrics, tools, named_graph)
    result = rohub.query_sparql_endpoint(query_string)
    if not result.empty:
        result["element_order"] = pd.to_numeric(
            result["element_order"], errors="coerce"
        )
        result["element_degree"] = pd.to_numeric(
            result["element_degree"], errors="coerce"
        )
        filtered_result = result[
            (result["element_order"] == 2) & (result["element_degree"] == 2)
        ]
        rows = [
            [
                float(row["element_size"]),
                float(row["max_von_mises_stress_gauss_points"]),
                row["tool_name"],
            ]
            for _, row in filtered_result.iterrows()
        ]
        data.extend(rows)

data.sort(key=lambda row: row[0], reverse=False)

analyzer.plot_provenance_graph(
    data=data,
    x_axis_label="Element Size",
    y_axis_label="Max Von Mises Stress Nodes Gauss Points ",
    x_axis_index=0,
    y_axis_index=1,
    group_by_index=2,
    title="Element Size vs Max Von Mises Stress Nodes Gauss Points  \n element-order = 2 , element-degree = 2 ",
)